# JOIN

In [1]:
import sys
import sqlite3
sys.path.append('tests')
from tests.test_join import *

In [2]:

conn = sqlite3.connect('consultoria.db')

conn.execute("DROP TABLE IF EXISTS projetos")
conn.execute("DROP TABLE IF EXISTS clientes")
conn.execute("DROP TABLE IF EXISTS consultores")
conn.execute("DROP TABLE IF EXISTS feedbacks")
conn.execute("DROP TABLE IF EXISTS alocacoes")

conn.execute("PRAGMA foreign_keys = ON;")

print("✅ Conectado ao banco de dados 'consultoria.db'")
print("✅ Chaves estrangeiras habilitadas")

backup_conn = None

def backup_db():
    """Cria um backup do banco em memória"""
    global backup_conn
    backup_conn = sqlite3.connect(':memory:')
    conn.backup(backup_conn)
    print("📁 Backup criado")

def restore_db():
    """Restaura o banco a partir do backup"""
    global backup_conn
    if backup_conn:
        backup_conn.backup(conn)
        print("🔄 Banco restaurado do backup")
    else:
        print("❌ Nenhum backup disponível")

def validate_and_execute(query, test_function):
    """Valida query em ambiente isolado antes de executar no banco principal"""
    if not query or not query.strip():
        print("❌ Query vazia! Escreva sua query SQL antes de executar.")
        return False
    
    backup_db()
    
    try:
        test_conn = sqlite3.connect(':memory:')
        test_conn.execute("PRAGMA foreign_keys = ON;")
        
        for linha in conn.iterdump():
            if not linha.startswith('BEGIN') and not linha.startswith('COMMIT'):
                try:
                    test_conn.execute(linha)
                except:
                    pass  
        

        test_conn.execute(query)
        test_conn.commit()

        conn.execute(query)
        conn.commit()
        
        success = test_function(conn)
        
        if success:
            print("✅ Query executada e validada com sucesso!")
            return True
        else:
            restore_db()
            print("💡 Query executada mas resultado incorreto. Banco restaurado, tente novamente!")
            return False
            
    except Exception as e:
        print(f"❌ Erro na query: {str(e)}")
        print("💡 Corrija a sintaxe e tente novamente!")
        return False
    finally:
        if 'test_conn' in locals():
            test_conn.close()

def test_and_rollback(test_function):
    """Mantido para compatibilidade - executa teste simples"""
    success = test_function(conn)
    if not success:
        print("💡 Execute a query correta primeiro!")
    return success

def validate_and_execute_inserts(insert_queries, test_function):
    """Valida e executa múltiplas queries INSERT com rollback automático"""
    if not insert_queries or not insert_queries.strip():
        print("❌ Queries vazias! Escreva seus INSERTs antes de executar.")
        return False
    backup_db()
    
    try:
        queries = [q.strip() for q in insert_queries.strip().split(';') if q.strip()]
        
        if not queries:
            print("❌ Nenhuma query válida encontrada!")
            return False
        
        for query in queries:
            conn.execute(query)
        conn.commit()

        success = test_function(conn)
        
        if success:
            print("✅ Dados inseridos e validados com sucesso!")
            return True
        else:
            restore_db()
            print("💡 Dados inseridos mas resultado incorreto. Banco restaurado, tente novamente!")
            return False
            
    except Exception as e:
        restore_db()
        print(f"❌ Erro ao inserir dados: {str(e)}")
        print("💡 Corrija a sintaxe e tente novamente!")
        return False

print("\n🛠️ Sistema de backup/restore configurado!")
print("Agora vamos começar a criar nossas tabelas...")
def validate_and_execute_select(sql_query, test_function):
    """Valida e executa query SELECT com rollback automático"""
    if not sql_query or not sql_query.strip():
        print("❌ Query vazia! Escreva sua query SQL antes de executar.")
        return False
    
    backup_db()
    
    try:
        success = test_function(sql_query, conn)
        
        if success:
            print("✅ Query SELECT validada com sucesso!")
            df = pd.read_sql_query(sql_query, conn)
            print("\n📊 Resultado da sua query:")
            print(df.to_string(index=False))
            return True
        else:
            restore_db()
            print("💡 Query executada mas resultado incorreto. Banco restaurado, tente novamente!")
            return False
            
    except Exception as e:
        restore_db()
        print(f"❌ Erro na query: {str(e)}")
        print("💡 Corrija a sintaxe e tente novamente!")
        return False



✅ Conectado ao banco de dados 'consultoria.db'
✅ Chaves estrangeiras habilitadas

🛠️ Sistema de backup/restore configurado!
Agora vamos começar a criar nossas tabelas...




### 1. INNER JOIN  

Retorna apenas os registros que têm correspondência nas duas tabelas.  

📌 **Sintaxe**:
```sql
SELECT colunas
FROM tabela1
INNER JOIN tabela2
    ON tabela1.coluna = tabela2.coluna;
```


# Agora vamos praticar! 

In [10]:
# TODO: Listar o título de cada projeto e o nome do cliente correspondente
# Dica: Use INNER JOIN para conectar as tabelas PROJETOS e CLIENTES
sql_inner_join_1 = """
    SELECT PROJETOS.titulo,CLIENTES.nome AS nome_cliente FROM PROJETOS INNER JOIN CLIENTES ON PROJETOS.cliente_id = CLIENTES.id
"""

validate_and_execute_select(sql_inner_join_1, test_inner_join)

📁 Backup criado
✅ INNER JOIN validado com sucesso! Resultado confere com o esperado.
✅ Query SELECT validada com sucesso!

📊 Resultado da sua query:
       titulo        nome_cliente
        ERP I  TechCorp Solutions
 MARKETING II     Verde Agro Ltda
LOGISTICA III   FastLogistic S.A.
  EDUCACAO IV      EduCare Ensino
     VENDAS V MetalMax Indústrias
FINANCEIRO VI  TechCorp Solutions


True

In [13]:
#TODO: listar projeto, cliente e nota de todos os projetos avaliados, organizados por notas decrescente.
# Dica: Use GROUP BY para agrupar por nota e titulo
sql_inner_join_medium = """
 SELECT
    PROJETOS.titulo,
    CLIENTES.nome AS nome_cliente,
    FEEDBACKS.nota
FROM PROJETOS
INNER JOIN CLIENTES
    ON PROJETOS.cliente_id = CLIENTES.id
INNER JOIN FEEDBACKS
    ON FEEDBACKS.projeto_id = PROJETOS.id
GROUP BY
    PROJETOS.titulo,
    CLIENTES.nome,
    FEEDBACKS.nota
ORDER BY
    FEEDBACKS.nota DESC
"""
# Rodar:
validate_and_execute_select(sql_inner_join_medium, test_inner_join_medium)

📁 Backup criado
✅ INNER JOIN (médio) validado com sucesso!
✅ Query SELECT validada com sucesso!

📊 Resultado da sua query:
       titulo        nome_cliente  nota
  EDUCACAO IV      EduCare Ensino     5
        ERP I  TechCorp Solutions     5
FINANCEIRO VI  TechCorp Solutions     4
 MARKETING II     Verde Agro Ltda     4
     VENDAS V MetalMax Indústrias     3


True

In [17]:
#TODO:Para cada cliente e frente (ENG/BUS/DIR), some as horas alocadas em projetos que têm feedback com nota ≥ 4 
# e calcule também a média da nota desses projetos e a quantidade de projetos distintos considerados.
# Dica: Use AVG e COUNT DISTINCT para calcular a média e a contagem de projetos distintos
sql_inner_join_hard = """
    SELECT
    CLIENTES.nome AS nome_cliente,
    CONSULTORES.frente,
    SUM(ALOCACOES.horas_trabalhadas) AS total_horas,
    AVG(FEEDBACKS.nota) AS media_nota,
    COUNT(DISTINCT PROJETOS.id) AS qtd_projetos
FROM CLIENTES
INNER JOIN PROJETOS
    ON PROJETOS.cliente_id = CLIENTES.id
INNER JOIN FEEDBACKS
    ON FEEDBACKS.projeto_id = PROJETOS.id
INNER JOIN ALOCACOES
    ON ALOCACOES.projeto_id = PROJETOS.id
INNER JOIN CONSULTORES
    ON CONSULTORES.id = ALOCACOES.consultor_id
WHERE FEEDBACKS.nota >= 4
GROUP BY
    CLIENTES.nome,
    CONSULTORES.frente
ORDER BY
    CLIENTES.nome,
    CONSULTORES.frente;

"""

validate_and_execute_select(sql_inner_join_hard, test_inner_join_hard)

📁 Backup criado
✅ INNER JOIN (hard, sem HAVING) validado com sucesso!
✅ Query SELECT validada com sucesso!

📊 Resultado da sua query:
      nome_cliente frente  total_horas  media_nota  qtd_projetos
    EduCare Ensino    BUS        200.0         5.0             1
TechCorp Solutions    DIR         80.0         5.0             1
TechCorp Solutions    ENG        210.5         4.5             2
   Verde Agro Ltda    BUS        205.5         4.0             1


True

### 2. LEFT JOIN  

Retorna **todos** os registros da tabela à esquerda (primeira tabela) e os registros correspondentes da tabela à direita. Se não houver correspondência, retorna NULL para as colunas da tabela direita.

📌 **Sintaxe**:
```sql
SELECT colunas
FROM tabela_esquerda
LEFT JOIN tabela_direita
    ON tabela_esquerda.coluna = tabela_direita.coluna;
```

📊 **Diferença do INNER JOIN**:
- **INNER JOIN**: Só registros que têm correspondência nas duas tabelas
- **LEFT JOIN**: Todos da esquerda + correspondências da direita (NULL se não houver)

💡 **Exemplo prático**: No LEFT JOIN entre CLIENTES e PROJETOS:
- Todos os clientes aparecerão no resultado
- Clientes com projetos: mostram os títulos dos projetos
- Clientes sem projetos: mostram NULL na coluna titulo

In [19]:
# TODO: Listar TODOS os consultores e seus projetos (incluindo consultores sem projetos)
# Dica: Use LEFT JOIN para garantir que todos os consultores apareçam


sql_left_join_1 = """
    SELECT
    PROJETOS.titulo AS titulo,
    CONSULTORES.nome AS nome_consultor
FROM CONSULTORES
LEFT JOIN ALOCACOES
    ON ALOCACOES.consultor_id = CONSULTORES.id
LEFT JOIN PROJETOS
    ON PROJETOS.id = ALOCACOES.projeto_id
ORDER BY
    CONSULTORES.nome,
    PROJETOS.titulo;

"""

# Quando terminar sua query, execute a validação:
validate_and_execute_select(sql_left_join_1, test_left_join)

📁 Backup criado
✅ LEFT JOIN validado com sucesso! Todos os projetos incluídos, mesmo sem consultores.
✅ Query SELECT validada com sucesso!

📊 Resultado da sua query:
       titulo nome_consultor
        ERP I      Ana Silva
FINANCEIRO VI      Ana Silva
LOGISTICA III      Ana Silva
 MARKETING II  Carlos Santos
     VENDAS V  Carlos Santos
  EDUCACAO IV  Fernanda Lima
 MARKETING II  Fernanda Lima
LOGISTICA III     João Costa
        ERP I Maria Oliveira
     VENDAS V Maria Oliveira


True

In [21]:
# TODO: Listar todos os clientes e seus projetos (se existirem), e ordene por nome do cliente e título do projeto

sql_left_join_2 = """
    SELECT
    CLIENTES.nome AS nome_cliente,
    PROJETOS.titulo AS titulo
FROM CLIENTES
LEFT JOIN PROJETOS
    ON PROJETOS.cliente_id = CLIENTES.id
ORDER BY
    CLIENTES.nome,
    PROJETOS.titulo;

"""


validate_and_execute_select(sql_left_join_2, test_left_join_medium)

📁 Backup criado
✅ LEFT JOIN (médio) validado com sucesso!
✅ Query SELECT validada com sucesso!

📊 Resultado da sua query:
       nome_cliente        titulo
     EduCare Ensino   EDUCACAO IV
  FastLogistic S.A. LOGISTICA III
MetalMax Indústrias      VENDAS V
 TechCorp Solutions         ERP I
 TechCorp Solutions FINANCEIRO VI
    Verde Agro Ltda  MARKETING II


True

In [25]:
# TODO: Para cada cliente, listar:
# projetos_total: nº de projetos
# projetos_com_feedback: nº de projetos que têm ao menos 1 feedback (qualquer nota)
# projetos_nota_ge_4: nº de projetos com nota ≥ 4
# media_nota: média das notas dos projetos com feedback
# total_horas: soma de horas alocadas em todos os seus projetos

#Dicas:Use LEFT JOIN a partir de clientes → projetos → feedbacks e alocacoes
# Agrupe por cliente
# Ordene por nome_cliente ASC
# Não filtre clientes sem feedback (devem aparecer; media_nota pode ser NULL)

sql_left_join_hard = """
    WITH por_projeto AS (
    SELECT
        PROJETOS.id AS projeto_id,
        PROJETOS.cliente_id,

        AVG(FEEDBACKS.nota) AS nota_projeto,
        SUM(ALOCACOES.horas_trabalhadas) AS horas_projeto

    FROM PROJETOS
    LEFT JOIN FEEDBACKS
        ON FEEDBACKS.projeto_id = PROJETOS.id
    LEFT JOIN ALOCACOES
        ON ALOCACOES.projeto_id = PROJETOS.id
    GROUP BY
        PROJETOS.id,
        PROJETOS.cliente_id
)

SELECT
    CLIENTES.nome AS nome_cliente,

    COUNT(DISTINCT por_projeto.projeto_id) AS projetos_total,

    COUNT(DISTINCT CASE
        WHEN por_projeto.nota_projeto IS NOT NULL THEN por_projeto.projeto_id
    END) AS projetos_com_feedback,

    COUNT(DISTINCT CASE
        WHEN por_projeto.nota_projeto >= 4 THEN por_projeto.projeto_id
    END) AS projetos_nota_ge_4,

    AVG(por_projeto.nota_projeto) AS media_nota,

    SUM(por_projeto.horas_projeto) AS total_horas

FROM CLIENTES
LEFT JOIN por_projeto
    ON por_projeto.cliente_id = CLIENTES.id

GROUP BY
    CLIENTES.nome

ORDER BY
    nome_cliente ASC;

"""


# 3) rode o validador corretamente
validate_and_execute_select(sql_left_join_hard, test_left_join_hard)

📁 Backup criado
✅ LEFT JOIN (difícil) validado com sucesso!
✅ Query SELECT validada com sucesso!

📊 Resultado da sua query:
       nome_cliente  projetos_total  projetos_com_feedback  projetos_nota_ge_4  media_nota  total_horas
     EduCare Ensino               1                      1                   1         5.0        200.0
  FastLogistic S.A.               1                      0                   0         NaN        225.5
MetalMax Indústrias               1                      1                   0         3.0        145.5
 TechCorp Solutions               2                      2                   2         4.5        290.5
    Verde Agro Ltda               1                      1                   1         4.0        205.5


True